In [1]:
import argparse
import sys
sys.path.append('../src')
import loader
import rnn_encoder_decoder
from torchtext import data
import torch
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from elmoformanylangs import Embedder

In [3]:
chinese_embedder = Embedder('/scratch/jp4989/elmo/179')

2018-11-15 18:16:54,693 INFO: char embedding size: 15889
2018-11-15 18:16:55,507 INFO: word embedding size: 140384
2018-11-15 18:17:01,458 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(140384, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(15889, 50, padding_idx=15886)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out

In [ ]:
# FileNotFoundError: [Errno 2] No such file or directory: 
# '/Users/yijialiu/work/projects/conll2018/models/word_elmo/cnn_50_100_512_4096_sample.json'
vietnamese_embedder = Embedder('/scratch/jp4989/elmo/178')
# FileNotFoundError: [Errno 2] No such file or directory: 
# '/Users/yijialiu/work/projects/conll2018/models/word_elmo/cnn_50_100_512_4096_sample.json'
english_embedder = Embedder('/scratch/jp4989/elmo/144')

In [8]:
simplified_chinese_embedder = Embedder('/scratch/jp4989/elmo/zhs.model')

2018-11-15 19:08:36,182 INFO: char embedding size: 6169
2018-11-15 19:08:36,583 INFO: word embedding size: 71222
2018-11-15 19:08:40,127 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_feat

In [9]:
# MUST USE GPU (?)
parser = rnn_encoder_decoder.rnn_encoder_decoder_argparser()
args = parser.parse_args([])
#args.cpu = True
args.split_chinese_into_characters = False

In [22]:
train_data, val_data, test_data, src, trg = loader.load_data(args)

Most common source vocabs: [('的', 272264), ('我', 83174), ('是', 71290), ('我们', 68696), ('在', 55727), ('了', 52197), ('你', 40824), ('这', 36049), ('一个', 33590), ('他们', 25256)]
Source vocab size: 47127
Most common english vocabs: [('the', 178698), ('and', 128020), ('to', 106119), ('of', 98919), ('a', 90534), ('that', 80454), ('i', 68877), ('in', 66951), ('it', 64198), ('you', 61161)]
English vocab size: 28791


In [15]:
# WRONG WAY! this is embedding each character
tokenized_itos = chinese_embedder.sents2elmo(src.vocab.itos)

2018-11-15 19:12:41,640 INFO: 737 batches, avg len: 4.5
2018-11-15 19:12:42,191 INFO: Finished 1000 sentences.
2018-11-15 19:12:42,674 INFO: Finished 2000 sentences.
2018-11-15 19:12:43,100 INFO: Finished 3000 sentences.
2018-11-15 19:12:43,519 INFO: Finished 4000 sentences.
2018-11-15 19:12:43,948 INFO: Finished 5000 sentences.
2018-11-15 19:12:44,358 INFO: Finished 6000 sentences.
2018-11-15 19:12:44,827 INFO: Finished 7000 sentences.
2018-11-15 19:12:45,209 INFO: Finished 8000 sentences.
2018-11-15 19:12:45,633 INFO: Finished 9000 sentences.
2018-11-15 19:12:46,095 INFO: Finished 10000 sentences.
2018-11-15 19:12:46,512 INFO: Finished 11000 sentences.
2018-11-15 19:12:46,937 INFO: Finished 12000 sentences.
2018-11-15 19:12:47,433 INFO: Finished 13000 sentences.
2018-11-15 19:12:47,851 INFO: Finished 14000 sentences.
2018-11-15 19:12:48,280 INFO: Finished 15000 sentences.
2018-11-15 19:12:48,696 INFO: Finished 16000 sentences.
2018-11-15 19:12:49,117 INFO: Finished 17000 sentences.
2

In [26]:
tokenized_itos[0].shape

(5, 1024)

In [ ]:
# WRONG! unk eos should now be oov? and bos
tokenized_itos_ver2 = chinese_embedder.sents2elmo([src.vocab.itos], output_layer=0)